In [1]:
cd ..

/gpfs01/bethge/home/oeberle/Scripts/deepgaze


In [2]:
from __future__ import print_function, division, unicode_literals
#import TotalSaliency as TS
import numpy as np
import itertools
import scipy
import matplotlib.pyplot as plt
import zipfile
%matplotlib inline
from boltons.fileutils import mkdir_p
from tqdm import tqdm
import shutil
import warnings
import glob
import itertools
import pysaliency
from pysaliency import get_FIGRIM
from baseline_models import BaselineModel, ShapeIndependentBaselineModel
import cPickle as pickle
import theano

/usr/local/lib/python2.7/dist-packages/matplotlib/cbook.py:128: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the matplotlib toolkit finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5105)
/usr/local/lib/python2.7/dist-packages/optpy/ipopt_wrapper.py:15: UserWarning: Could not import ipopt. Probably cyipopt is not installed
  warnings.warn('Could not import ipopt. Probably cyipopt is not installed')


In [3]:
#Get Figrim datased
location = '/gpfs01/bethge/home/oeberle/Scripts/deepgaze/'
figrim_stimuli, figrim_fixations= get_FIGRIM(location=location)

def compute_memorability_scores(image_dict):
    HR = float(image_dict['hits']) / (float(image_dict['hits'])+ float(image_dict['misses']))
    FAR =  float(image_dict['false_alarms']) / (float(image_dict['false_alarms'])+ float(image_dict['correct_rejections']))
    ACC = (float(image_dict['hits']) + float(image_dict['correct_rejections']))/ (float(image_dict['hits'])+ float(image_dict['misses']) + float(image_dict['false_alarms'])+ float(image_dict['correct_rejections']))
    return {'HR': HR, 'FAR': FAR, 'ACC': ACC}



def get_memory_scores(csv_url, save_dir, save_name):
    import csv
    import requests
    import cPickle as pickle
    import collections
    import urllib

    try:
        data = pickle.load(open(save_dir + save_name + '.p', 'r'))
    except IOError:     
        testfile = urllib.URLopener()
        testfile.retrieve(csv_url, save_dir  + save_name + '.csv')
        
        
        csvfile = open( save_dir  + save_name + '.csv', 'r')
        jsonfile = open(save_dir  + save_name + '.json', 'w')


        fieldnames = ('image number', 'scene category', 'filename', 'hits', 'misses', 'false_alarms', 'correct_rejections')
        reader = csv.DictReader(csvfile, fieldnames)
        data = collections.OrderedDict()
        for i,row in enumerate(reader):
            if i == 0:
                pass
            else:
                score_dict = compute_memorability_scores(row)
                row.update(score_dict)
                data.update({row['image number']:row})
        pickle.dump(data,open(save_dir + save_name + '.p', 'w'))
    return data

def get_image_from_filename(scene_category, filename, root_dir):
    '''
    Use np.asarray(jpgfile) to get rgb data 
    '''
    import Image


    img_dir =  root_dir + '/Targets/' + scene_category + '/' + filename
    try:
        jpgfile = Image.open(img_dir)
        typ = 'target'
    except IOError:
        img_dir =  root_dir + '/Fillers/' + scene_category + '/' + filename
        try:
            jpgfile = Image.open(img_dir)
            typ = 'filler'
        except IOError:
            typ ='missing'
            jpgfile = None
            
    return typ, jpgfile, np.asarray(jpgfile)


# Get saliency model
import yaml
import os
import dill

from train_deep_gaze import get_dataset, load_experiment, load_parameters
from scipy.ndimage import zoom
from boltons.cacheutils import LRU

class DeepGazeModel(pysaliency.Model):
    """
    Create and provide a set of DeepSal objects for different image sizes.
    """
    def __init__(self, directory, crossval_id = 0, centerbias_model=None, network_cache=None, **kwargs):
        super(DeepGazeModel, self).__init__(**kwargs)
        self.directory = directory
        self.crossval_id = crossval_id
        if network_cache is None:
            self.networks = LRU(max_size=2)
        else:
            print("Reusing network cache")
            self.networks = network_cache
        self.centerbias_model = centerbias_model
        self.params = load_parameters(self.directory, crossval_id = self.crossval_id)

    def load_network(self, size):
        new_network = load_experiment(self.directory, size, crossval_id = self.crossval_id)
        new_network.prepared_output_cache.size = 4
        self.networks[size] = new_network

        return self.networks[size]

    def prepare_centerbias(self, stimulus):
        
        size = (stimulus.shape[0], stimulus.shape[1])
        network = self.get_network(size)
        
        centerbias = self.centerbias_model.log_density(stimulus)

        y_size = network.salmap_shape[0]
        x_size = network.salmap_shape[1]

        image_area = (x_size*y_size)/(size[0]*size[1])

        factor_x = x_size/centerbias.shape[1]
        factor_y = y_size/centerbias.shape[0]
        #print('shapes', size, centerbias.shape)
        #print('factors', factor_x, factor_y)

        centerbias = zoom(centerbias, [factor_y, factor_x], order=1, mode='nearest')
        centerbias_density = np.exp(centerbias)
        centerbias -= (np.log(centerbias_density.sum()) - np.log(image_area))
        #print "This centre bias:", np.exp(centerbias).sum()
        np.testing.assert_allclose(np.exp(centerbias).sum(), image_area)

        return centerbias

    def get_network(self, shape):
        if shape not in self.networks:
            self.load_network(shape)
        network = self.networks[shape]
        for p, v in zip(network.params, self.params):
            p.set_value(v)

        return network

    def _log_density(self, stimulus):
        """Generate saliencymap for given image using given centerbias
        centerbias_template == None falls back to default centerbias template
        """
        shape = (stimulus.shape[0], stimulus.shape[1])
        network = self.get_network(shape)
        centerbias = self.prepare_centerbias(stimulus)
        #print(centerbias.shape)
        
        network_input = network.prepare_network_input(self.ensure_color(stimulus).copy())
        
        saliency_map = network.generate_log_densities(network_input.astype(theano.config.floatX),
                                                      centerbias.astype(theano.config.floatX))
        if saliency_map.shape != shape:
            print("WARNING! WRONG SHAPE!!! {} != {}. Resizing".format(saliency_map.shape, shape))
            saliency_map = zoom(saliency_map, [shape[0]/saliency_map.shape[0], shape[1]/saliency_map.shape[1]])
            
            saliency_map -= np.log(np.sum(np.exp(saliency_map)))
            
            assert saliency_map.shape == shape
        return saliency_map

    def ensure_color(self, image):
        if image.ndim == 2:
            return np.dstack((image, image, image))
        return image

    
def load_val_deepgaze_model(directory, centerbias_model=None, network_cache=None, **kwargs):
    config = yaml.load(open(os.path.join(directory, 'config.yaml')))
    baseline_path = config['centerbias'].get('disk_cache', '.')
    crossval_id = 0
    #val_baseline_name = os.path.join(directory, baseline_path, 'baseline_val_{}.pydat'.format(crossval_id))
    #val_baseline = dill.load(open(val_baseline_name))
    #val_baseline = dill.load(open('deepgaze2_2/experiments/cache/MIT1003_twosize-10-0-rand/'))
    
    return DeepGazeModel(directory, centerbias_model=centerbias_model, network_cache=network_cache, **kwargs)


def glue_images(imga, imgb, offset = 500):
    """
    Combines two color image ndarrays side-by-side.
    """
    ha,wa = imga.shape[:2]
    hb,wb = imgb.shape[:2]
    max_height = np.max([ha, hb])
    #if max_height % 2. != 0:
    #    max_height = max_height + 1
    offset = np.max([wa,wb])
    total_width = wa+wb+offset
   
    #new_img[:ha,:wa]=imga
    #new_img[:hb,wa+offset:wa+wb+offset]=imgb
    hdiff = int(np.ceil((ha-hb)/2.))
    
    #Adding borders
    max_height = max_height + 2*np.abs(hdiff)
    total_width = total_width + 2*np.abs(hdiff)
    new_img = np.zeros(shape=(max_height, total_width, 3), dtype=np.uint8)

    #print(hdiff)
    if hdiff >= 0: # imgb smaller than imga
        new_img[hdiff:ha+hdiff,hdiff:wa+hdiff]=imga
        new_img[2*hdiff:hb+2*hdiff,wa+offset+hdiff:wa+wb+offset+hdiff]=imgb
    elif hdiff < 0: # imga smaller than imgb
        hdiff = np.abs(hdiff)
        new_img[2*hdiff:ha+2*hdiff,hdiff:wa+hdiff]=imga
        new_img[hdiff:hb+hdiff,wa+offset+hdiff:wa+wb+offset+hdiff]=imgb
        
    center = wa + hdiff + int(np.ceil(offset/2.))
    return new_img, center


class TotalSaliency(object):
    def __init__(self, img1, img2):
        self.img1 = img1
        self.img2 = img2
    def __call__(self):    
        im_glued, center = glue_images(self.img1, self.img2)
        log_density = uniform_dg_model.log_density(stimulus=im_glued)
        #print(log_density.shape)
        d_left, d_right = self.get_total_saliency(log_density, center)
        return log_density, d_left, d_right
    
    def get_total_saliency(self, log_density,center):
        density = np.exp(log_density)
        d_left = np.sum(density[:,:center])
        d_right = np.sum(density[:, center:])
        return d_left, d_right

In [4]:
base_directory = 'TrainedModels/experiment0010_SALICONMIT2_MIT2_rand'
#target_directory = 'model_caches/deep_gaze_2/experiment0010_SALICONMIT2_MIT2_rand/'
config = yaml.load(open(os.path.join(base_directory, 'crossval_10-0', 'config.yaml')))


baseline = ShapeIndependentBaselineModel([figrim_stimuli], [figrim_fixations], config['centerbias']['bandwith'], regularization=config['centerbias']['regularization'])

cb_dg_models = []
uniform_dg_models = []
for crossval in sorted(glob.glob(os.path.join(base_directory, 'crossval*'))):
    if uniform_dg_models:
        network_cache = uniform_dg_models[0].networks
    else:
        network_cache = None
    
    uniform_dg_model = load_val_deepgaze_model(crossval, centerbias_model=pysaliency.UniformModel(),
                                               network_cache=network_cache)
    uniform_dg_models.append(uniform_dg_model)
    
    cb_dg_model = load_val_deepgaze_model(crossval, centerbias_model=baseline,
                                          network_cache=uniform_dg_models[-1].networks)
    cb_dg_models.append(cb_dg_model)
    
    
    #config = yaml.load(open(os.path.join(crossval, 'config.yaml')))
    #dataset, f = get_dataset(config['optimization']['val_dataset'], location='test_datasets')
    #for s in dataset:
    #    dg_model.log_density(s)
    #    clear_output()
uniform_dg_model = pysaliency.MixtureModel(uniform_dg_models)
cb_dg_model = pysaliency.MixtureModel(cb_dg_models)

data = get_memory_scores('http://figrim.mit.edu/within_category.csv',location + 'FIGRIM/Memory/', 'within_category')
#data = get_memory_scores('http://figrim.mit.edu/across_category.csv',location + 'FIGRIM/Memory/', 'across_category')

2793 (100.0%)
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache
Reusing network cache


In [ ]:
# Compute Total Saliency for image
typ_ref,_,reference_im = get_image_from_filename('bathroom', 'sun_aanjcbzdvtvblsdy.jpg', location + 'FIGRIM/Stimuli/' )


data_copy = data.copy()
for d in data_copy.itervalues():
    typ_comp,comp_im_obj,comparison_im = get_image_from_filename(d['scene category'],  d['filename'], location + 'FIGRIM/Stimuli/')
    try:
        print(d['image number'])

        open('/gpfs01/bethge/home/oeberle/Scripts/deepgaze/OliverPlayground/Memory_within_Scenes/memory_data_{}.p'.format(d['image number']), 'r')

    except IOError:
        print('Creating: ',d['image number'])
        if typ_comp == 'target':
            TSG = TotalSaliency(reference_im, comparison_im)
            log_density, d_left, d_right = TSG.__call__()
            d['TS'] = np.log(d_right/d_left)
            d['log_density'] = log_density
            d['type'] = typ_comp
        else: 
            d['type'] = typ_comp
        pickle.dump(d, open('/gpfs01/bethge/home/oeberle/Scripts/deepgaze/OliverPlayground/Memory_within_Scenes/memory_data_{}.p'.format(d['image number']), 'w'))
pickle.dump(data, open('/gpfs01/bethge/home/oeberle/Scripts/deepgaze/OliverPlayground/Memory_within_Scenes/memory_data_all.p', 'w'))

#Downsample
#reference_im = scipy.misc.imresize(reference_im, (500,100), interp='bilinear')
#comparison_im = scipy.misc.imresize(comparison_im, (98,100), interp='bilinear')
#im_glued, center = TS.glue_images(reference_im, comparison_im, 500)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
